In [1]:
%load_ext autoreload
%autoreload 2

In [39]:
import pandas as pd
tickers = pd.read_csv("data/stock_etf_tickers.csv")
tickers = list(tickers.Ticker)

In [45]:
from src.DataCollector import DataCollector

dc = DataCollector(tickers)
rt = dc.get_returns()

[*********************100%***********************]  96 of 96 completed


In [51]:
from src.Portfolio import Portfolio

portfolio = Portfolio(rt)

portfolio.get_max_sharpe()

TypeError: unhashable type: 'dict'

In [34]:
set([x for x in tickers if tickers.count(x) > 1])

{'IWB', 'IWM'}

In [33]:
uniq, 

['MSFT',
 'AAPL',
 'NVDA',
 'AMZN',
 'GOOG',
 'META',
 'BRK.B',
 'AVGO',
 'TSLA',
 'WMT',
 'LLY',
 'JPM',
 'V',
 'UNH',
 'MA',
 'XOM',
 'PG',
 'JNJ',
 'CVX',
 'HD',
 'PFE',
 'BAC',
 'PEP',
 'CSCO',
 'ORCL',
 'INTC',
 'ADBE',
 'CRM',
 'MCD',
 'VZ',
 'T',
 'DIS',
 'NFLX',
 'CMCSA',
 'IBM',
 'QCOM',
 'MDT',
 'ABBV',
 'HON',
 'BA',
 'SBUX',
 'CAT',
 'GS',
 'AXP',
 'PYPL',
 'INTU',
 'MRNA',
 'ZM',
 'SHOP',
 'SQ',
 'VOO',
 'SPY',
 'IVV',
 'VTI',
 'QQQ',
 'VUG',
 'VEA',
 'VTV',
 'IEFA',
 'BND',
 'AGG',
 'IWF',
 'VWO',
 'IWM',
 'VB',
 'VYM',
 'EEM',
 'XLK',
 'XLF',
 'XLV',
 'XLE',
 'XLY',
 'XLI',
 'XLC',
 'XLU',
 'XLB',
 'IWB',
 'IWR',
 'IWN',
 'MTUM',
 'QUAL',
 'USMV',
 'VBR',
 'VO',
 'VOT',
 'SCHD',
 'IWD',
 'RSP',
 'ARKK',
 'ARKW',
 'ARKG',
 'IHI',
 'IBB',
 'XBI',
 'VGT',
 'SMH',
 'LQD',
 'SHY']